In [ ]:
# !aws s3 cp segments_cs_gb s3://monlam.ai.stt/stt_pecha_tools/ --recursive

In [2]:
group = "gb"
target_directory = f"segments_cs_{group}"

In [9]:
import os
def getTimeSpan(filename):
    filename = os.path.splitext(os.path.basename(filename))[0]
    try:
        if "_to_" in filename:
            start, end = filename.split("_to_")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)/1000
        else:
            start, end = filename.split("-")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)
    except Exception as err:
        print(f"filename is:'{filename}'. Could not parse to get time span.")
        return 0
    
import os
def getTimeSpan(filename):
    filename = os.path.splitext(os.path.basename(filename))[0]
    try:
        if "_to_" in filename:
            start, end = filename.split("_to_")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)/1000
        else:
            start, end = filename.split("-")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)
    except Exception as err:
        print(f"filename is:'{filename}'. Could not parse to get time span.")
        return 0
    
getTimeSpan("STT_AB00094_0057_290646_to_295860.mp3")
getTimeSpan("STT_AB00094_0057_290646_to_295860")

5.214

In [28]:
import os
rows = []
from transformers import pipeline
generator = pipeline(model="openpecha/wav2vec2_run8")

for root, _, files in os.walk(target_directory):
        for file in files:
            source_file_path = os.path.join(root, file)
            # print(f"{file} https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file}")
            opt = generator(f"/media/monlamai/FEEE31DEEE31903F/split/STT_CS/segments_cs_gb/{file}")
            print(opt)
            if "text" in opt:
                inf = opt['text']
            else:
                inf = ""
            rows.append([file, f"https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file}", inf , getTimeSpan(file)])

{'text': 'ཡ། ཤད་དེ་ལོ། སྐད་ཏོག་ཙི་ྱ་ས་རྒྱད་ཨ།'}
{'text': '་མི་གཅིགི་དལློའགོ།'}
{'text': 'རེ།། ཅི་དེ་ཡ་མེ། ཅིག་ྟ་འགྲོ་་་་ོ།'}
{'text': 'ཁྱོདྐྱིད་པ་ཟ་ཤེ་ཀི་མེ་།ཧའ་དེ་འཁྲུག་པ་འདྲ་བོ་ག་གི་རེད་ད་ར། ཨ་ནས་ཁྱོད་ར།ནང་ག་ར་ནས་་།ཨ་ནས།'}
{'text': 'སྔ་མ་རྗེས་ལཨ་ནས་བུ་གསེར་པོ་དེ་ཨེ་འདྲ་ལབ་ཀྱི་རེད་ད། ང་དུག་སློག་གསོ་བོ་ཟིག་རྒྱ་ཡིན། ཨ་ནས།ཨ་ནས་བུ་གཞན་པ་དེ་འདྲ་ལབ་ཀྱི་རེད་ད། ང་དུག་སལོག་དམར་བོ་མ་ཁྱེ་ཡིན་ག་རེ་བྱས་ས་ནས་ལབ་ཀྱི་རེད་དཱ། ཨ་ནས་ང་ཚོ་་རྗ་ྗ་།'}
{'text': 'ན་ག་བྱེས་སོང་ལབ་ཀྱ་རེད་ད། ང་པ། ཨ་ནས་བུ་མོ། སྔར་གྱོན་སྐྱོ་བུ་མོ་ཀ་ཅིག་སླེབས་ཀྱི་རེད་དཱ། ཨ་ནས་འོ་དེ་ཁོ་ྲུག་ཅ་ཅོག་ན་སྡུག་གི་རེད་ད་ར། འོ་དྲ་ལབ་ཀྱི་རེད་ད།'}
{'text': 'ཨཨོ། ཨ་ནས་ཕར་ཕྱུགས་་ལ་རཱ། ག་རེ།མི་སློང་ཀན་གཅིག་ཡོད་རེད་ད། ཨ་ནས་སྔོན་པོ་དེཁོ་ལ་བུ་མོ་སིར་ཀ་འདྲ་བོ་ཅིག་སྤྲད་ཀྱི་རེད་ད། ཨ་ནས་རྗེས་ལ་ཨེ་འདྲའས་ལབ་ཀྱི་རེད་ད།'}
{'text': 'ཁྱོད་རཱ། ག་རེ། འདི་འདུག་ག་ག་རེ།།'}
{'text': '་ནསའདི་ལ་ཁ་ལང་མང་པོ་ཟ་རཱ། ཨེ་ནས་རྗེས་ལ་ལས་ཀ་་བྱེད་ཨ་ལབ་ཀྱི་རེད་ད། ཨ་ནསཨོ་ལབ་ཀྱི་རེད་ད། ཨསགྷ་རི་ལ་སགུག་བསྡད་ཀྱི་རེད་ད། རྨོ་མོ་གཅིག་སྤོ་སཔོ་གཅིག་རེད་ད་རཱ། ག་རེ།'}
{'text': 'བུ་གཅིག་ཀ་ནས་ཨ

In [29]:
rows[0]

['STT_CS-D-2012-M-D-B09-6-R-02_0001_2141_to_5759.wav',
 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_CS-D-2012-M-D-B09-6-R-02_0001_2141_to_5759.wav',
 'ཡ། ཤད་དེ་ལོ། སྐད་ཏོག་ཙི་ྱ་ས་རྒྱད་ཨ།',
 3.618]

In [30]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference', 'audio_duration'])

In [31]:
df.head()

,file_name,url,inference,audio_duration
0,STT_CS-D-2012-M-D-B09-6-R-02_0001_2141_to_5759...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཡ། ཤད་དེ་ལོ། སྐད་ཏོག་ཙི་ྱ་ས་རྒྱད་ཨ།,3.618
1,STT_CS-D-2012-M-D-B09-6-R-02_0002_7346_to_1245...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,་མི་གཅིགི་དལློའགོ།,5.111
2,STT_CS-D-2012-M-D-B09-6-R-02_0003_12457_to_175...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,རེ།། ཅི་དེ་ཡ་མེ། ཅིག་ྟ་འགྲོ་་་་ོ།,5.111
3,STT_CS-D-2012-M-D-B09-6-R-02_0004_19854_to_291...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཁྱོདྐྱིད་པ་ཟ་ཤེ་ཀི་མེ་།ཧའ་དེ་འཁྲུག་པ་འདྲ་བོ་ག་...,9.329
4,STT_CS-D-2012-M-D-B09-6-R-02_0005_29183_to_385...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,སྔ་མ་རྗེས་ལཨ་ནས་བུ་གསེར་པོ་དེ་ཨེ་འདྲ་ལབ་ཀྱི་རེ...,9.329


In [32]:
df.to_csv(f"segments_cs_{group}_inf.csv", index=False)

In [25]:
opt

{'text': 'ཚོན་ཁང་ཀ་ཅི་ལྕགས་བསྡད་ཡོད་རེད། གཅིག་རཱ། ཨེ་ནས་འགྲོ་རེད་ད། རཱ། ཨ་ནས།'}